In [ ]:
import socket
import threading

SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((SERVER_HOST, SERVER_PORT))
server_socket.listen()

connected_clients = []

def handle_client(client_socket, client_address):
    print(f"New connection from {client_address}")

    try:
        while True:
            message = client_socket.recv(1024).decode()
            if not message:
                break

            broadcast_message(message, client_socket)

    except ConnectionResetError:
        print(f"Connection from {client_address} closed unexpectedly.")
    finally:
        client_socket.close()
        connected_clients.remove(client_socket)

def broadcast_message(message, sender_socket):
    for client_socket in connected_clients:
        if client_socket != sender_socket:
            try:
                client_socket.send(message.encode())
            except ConnectionResetError:
                connected_clients.remove(client_socket)

print(f"Chat server started on {SERVER_HOST}:{SERVER_PORT}")

while True:
    client_socket, client_address = server_socket.accept()
    connected_clients.append(client_socket)

    client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_thread.start()
